In [44]:
from pyspark.sql import SparkSession
from glob import glob
import os

In [45]:
spark = SparkSession.builder.getOrCreate()

In [46]:
# get all .txt file paths in NER 2016 dataset
# text_file_paths = sorted(glob("./data/NER2016-TestData-16-9-2016/**/*.txt", recursive=True))
text_file_paths = sorted(glob("./data/NER2016-TrainingData-3-3-2017-txt/**/*.txt", recursive=True))
origin = "train"

In [47]:
tag_names = ["B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "O"]

In [48]:
# read file and extract tags
def extract_and_reformat(text_file_paths):
    for file_counter, filepath in enumerate(text_file_paths):
        with open(filepath, "r", encoding="utf-8") as f:
            content = f.readline()
            output_file = open(f"./data/formatted_data/2016/{origin}/{str(file_counter).zfill(5)}.txt", "w", encoding="utf-8")

            line_counter = 0

            while content:
                if line_counter <= 3:
                    line_counter += 1
                    content = f.readline()
                    continue
                # get tags
                tags = content.split("\t")

                # if tag is not a word
                if tags[0] == '<s>' or tags[0] == "</s>":
                    tags = []
                else:
                    tags[-1] = tags[-1].strip()

                if tags and len(tags) >= 4:
                    words = tags[0]
                    ner_tag = tags[3]
                    output_file.write(f"{words} {ner_tag}\n")

                content = f.readline()
            
            # prepare for next increment
            output_file.close()

extract_and_reformat(text_file_paths)

In [49]:
input_file = "./data/NER2016-TestData-16-9-2016/Test data (16-9-2016) Column/104682.txt"

In [52]:
df = spark.read.text(text_file_paths, lineSep="\n")

In [53]:
for i in range(10):
    text = df.collect()[i]["value"].split("\t")
    print(text)
    print("----")

['<title>Năm ngày ở bệnh_viện.</title>\r']
----
['<editor>Vietlex team, 8-2016</editor>\r']
----
['-DOCSTART-\r']
----
['<s>', '', '', '', '\r']
----
['Bệnh_viện', 'N', 'B-NP', 'O', 'O\r']
----
['X', 'Ni', 'B-NP', 'O', 'O\r']
----
['mới', 'R', 'O', 'O', 'O\r']
----
['7', 'M', 'B-NP', 'O', 'O\r']
----
['g', 'Nu', 'B-NP', 'O', 'O\r']
----
['sáng', 'N', 'B-NP', 'O', 'O\r']
----
